# **Nettoyage des données**


In [ ]:
!pip install -r requirements.txt

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import statsmodels.graphics.tsaplots as tsaplots

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from scipy import stats #(pour le calcul des fonctions de répartition)
from scipy.signal import savgol_filter

from Codes.scrpits_visualisations import (
    tracer_evolution,
    camembert,
    comparer_periodes
)



In [2]:
#Fonction pour tester tous les ARIMA possibles (somme dans le TD 4 du cours de STL)
from itertools import product

def evaluate_all_models(data, pmax=5, qmax=5, max_lag=15):
    """
    Teste automatiquement tous les ARIMA(p,0,q) pour p <= pmax et q <= qmax
    et retient ceux qui passent les critères :
    - Dernier AR significatif
    - Dernier MA significatif
    - Résidus ≈ bruit blanc (test de Ljung-Box)
    
    Paramètres:
    - data (pd.Series): série temporelle
    - pmax (int): ordre max pour p
    - qmax (int): ordre max pour q
    - max_lag (int): nombre de lags pour le test Ljung-Box
    
    Retourne:
    - modèles_valides (list): couples (p,q) qui passent les critères
    """
    modèles_valides = []
    
    # Générer toutes les combinaisons (p, q)
    all_pq = list(product(range(pmax+1), range(qmax+1)))

    for (p, q) in all_pq:
        if p == 0 and q == 0:
            continue  # Ne pas tester ARIMA(0,0,0)

        print(f"\nTest du modèle ARIMA({p},0,{q})")
        try:
            # Estimation du modèle
            model = ARIMA(data, order=(p, 0, q))
            fit = model.fit()

            # Résumé des coefficients
            coef = fit.params
            std_errors = fit.bse
            t_stats = coef / std_errors
            p_values = 2 * (1 - stats.norm.cdf(np.abs(t_stats)))
            summary = pd.DataFrame({
                'Coef': coef,
                'Std Err': std_errors,
                't-stat': t_stats,
                'p-value': p_values
            })

            # Vérification des p-values du dernier AR et MA
            ar_key = f'ar.L{p}'
            ma_key = f'ma.L{q}'
            
            ar_significatif = (p > 0 and ar_key in summary.index and summary.loc[ar_key, 'p-value'] < 0.05) or (p == 0)
            ma_significatif = (q > 0 and ma_key in summary.index and summary.loc[ma_key, 'p-value'] < 0.05) or (q == 0)
            
            # Test Ljung-Box global
            ljung = acorr_ljungbox(fit.resid, lags=[max_lag], return_df=True)
            ljung_pvalue = ljung.iloc[0]['lb_pvalue']
            ljung_test_ok = ljung_pvalue > 0.05

            # Résumé
            print(f" - Dernier AR significatif: {ar_significatif}")
            print(f" - Dernier MA significatif: {ma_significatif}")
            print(f" - Ljung-Box p-value = {ljung_pvalue:.4f} -> {'OK' if ljung_test_ok else 'Problème'}")

            if ar_significatif and ma_significatif and ljung_test_ok:
                print(" Modèle retenu.")
                modèles_valides.append((p, q))
            else:
                print(" Modèle rejeté.")

        except Exception as e:
            print(f" Problème sur ARIMA({p},0,{q}): {e}")
    
    print("\nRésumé : modèles retenus")
    print(modèles_valides)

    return modèles_valides


In [3]:
#Fonction pour calculer les AIC et BIC des modèles retenus par la fonction précédente
def compute_aic_bic(data, modèles, d=0):
    """
    Calcule AIC et BIC pour une liste de modèles ARIMA(p,d,q) donnés.

    Paramètres:
    - data (pd.Series): série temporelle
    - modèles (list of tuples): liste des (p,q) retenus
    - d (int): degré de différenciation (par défaut 0)
    
    Retourne:
    - results (pd.DataFrame): tableau AIC et BIC par modèle
    """
    résultats = []

    for (p, q) in modèles:
        try:
            model = ARIMA(data, order=(p, d, q)).fit()
            résultats.append({
                'p': p,
                'd': d,
                'q': q,
                'AIC': model.aic,
                'BIC': model.bic
            })
        except Exception as e:
            print(f" Problème pour ARIMA({p},{d},{q}): {e}")

    résultats_df = pd.DataFrame(résultats)
    résultats_df = résultats_df.sort_values(by='BIC').reset_index(drop=True)

    print("\nTableau AIC et BIC :")
    print(résultats_df)

    return résultats_df


In [4]:
def backcast_arima(df, col, order, trend=None):
    """
    Backcasting : prédit les valeurs manquantes au début de la série,
    en laissant le choix du trend (intercept/drift).

    Args
    ----
    df     : pd.DataFrame contenant la colonne à backcaster.
    col    : str, nom de la colonne (avec NaN au début).
    order  : tuple (p, d, q), ordre ARIMA.
    trend  : {'n', 'c', 't', 'ct'} ou None
             - None  : 'n' si d>0, sinon 'c' (ancien comportement)
             - 'n'   : pas d’intercept
             - 'c'   : intercept en niveaux
             - 't'   : drift (constante dans les différences)
             - 'ct'  : intercept + drift

    Returns
    -------
    pd.Series des valeurs backcastées alignées sur les NaN initiales.
    """
    y = df[col]
    first_valid_ix = y.first_valid_index()
    if first_valid_ix is None:
        raise ValueError("La série est entièrement NaN.")
    n_missing = y.index.get_loc(first_valid_ix)

    # Partie observée et inversion
    y_obs = y.iloc[n_missing:]
    y_rev = y_obs.iloc[::-1]

    # Choix automatique du trend si non précisé
    p, d, q = order
    if trend is None:
        trend = 'n' if d > 0 else 'c'

    # Ajustement et forecast
    model = ARIMA(y_rev, order=order, trend=trend)
    res   = model.fit()
    fc_rev = res.forecast(steps=n_missing)

    # Ré-inversion et alignement
    backcast = pd.Series(
        fc_rev.values[::-1],
        index=y.iloc[:n_missing].index,
        name=f'backcast_{col}'
    )
    return backcast


In [5]:
# Importation des données
df = pd.read_excel("Données_produites/base_de_données_v1.xlsx")


# Indexation
df = df.set_index('Unnamed: 0')

#Fonction qui construit un vecteur contenant le nom des variables
def variables_pays(pays):
    variable_pays=[f'CPI_{pays}',f'PIB_{pays}',f'LT_IR_{pays}',f'ST_IR_{pays}',f'WH_{pays}',f'P_Growth_{pays}']
    return(variable_pays)



## Etape 1: Eliminer les variables ayant trop de valeurs manquantes <a class="anchor" id="partie1"></a>

In [ ]:
#On élimine l'année 1995 pour éviter de faire du "backcasting" du CPI de tous les pays
df_clean=df.copy()
df_clean=df_clean.drop(["1995-Q1","1995-Q2","1995-Q3","1995-Q4"])

#On élimine l'Estonie de notre dataset
Estonia=variables_pays("Estonia")
df_clean=df_clean.drop(Estonia, axis=1)
df_clean

In [ ]:
# Les données de l'OCDE pour le CPI de la Suisse (méthodologie proche à celle d'Eurostat)
cpi_suisse = pd.read_csv(r"Données_extraites\CPI_Suisse_OCDE.csv")
cpi_suisse = cpi_suisse.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE')
cpi_suisse = cpi_suisse.drop(cpi_suisse.index[:4])


#On recopie ces données dans df_clean
df_clean['CPI_Switzerland']= cpi_suisse['Switzerland'].values
df_clean


## Etape 2: détermination des modèles (ARMA et compagnie) pour nettoyer les données manquantes  <a class="anchor" id="partie2"></a>

In [ ]:
# Clean values: format time series
df_clean_TS=df_clean.copy()
df_clean_TS.index = pd.PeriodIndex(df_clean.index, freq='Q')  #conversion en PeriodIndex
df_clean_TS.index = df_clean_TS.index.to_timestamp() #Conversion en DateTime

df_clean_TS


### LT_IR <a class="anchor" id="partie21"></a>

#### LT_IR_Latvia<a class="anchor" id="partie211"></a>

In [ ]:
#Le code qui suit sert uniquement à visualiser les ST
# syntaxe: tracer_evolution(df, columns=None, xlabel= None, ylabel=None, start_date=None, end_date=None, log=False, base=None, lissage=False):
tracer_evolution(df_clean_TS,["LT_IR_Latvia"], xlabel ="Période",ylabel = "Taux d'intérêt- court terme")


In [ ]:
#Pour le moment la série ne semble avoir ni tendance ni saisonalité, essayons d'appliquer un test ADF
result = adfuller(df_clean_TS["LT_IR_Latvia"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

#La série n'est pas stationnaire
#essayons de faire une différenciation

In [11]:
LT_IR_Latvia_diff=df_clean_TS["LT_IR_Latvia"].diff()
#LT_IR_Latvia_diff.plot(title="Taux d'intérêt Lettonie différencié", figsize=(12, 4))

In [ ]:
#Résult
result = adfuller(LT_IR_Latvia_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
#On rejette à 5% l'hypothèse de non stationarité


In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR Latvia")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR Latvia")
plt.show()


In [ ]:
LT_IR_Latvia_models=evaluate_all_models(LT_IR_Latvia_diff.dropna(), 3, 5)

In [ ]:
compute_aic_bic(LT_IR_Latvia_diff,LT_IR_Latvia_models,0)

In [ ]:
#On essaie d'appliquer la fonction précédente
#LT_IR_Latvia_pred=backcast_arima(df_clean_TS,"LT_IR_Latvia",(2,1,4))
LT_IR_Latvia_pred=backcast_arima(df_clean_TS, "LT_IR_Latvia", (2,1,4), trend='t')
LT_IR_Latvia_pred_final = LT_IR_Latvia_pred.combine_first(df_clean_TS["LT_IR_Latvia"])
LT_IR_Latvia_pred_final



# Si LT_IR_Latvia_pred_final est un pd.Series indexé en datetime :
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Latvia_pred_final.index, 
         LT_IR_Latvia_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Latvia — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()



#### LT_IR_Lithuania<a class="anchor" id="partie212"></a>

In [ ]:
#Le code qui suit sert uniquement à visualiser les ST
# syntaxe: tracer_evolution(df, columns=None, xlabel= None, ylabel=None, start_date=None, end_date=None, log=False, base=None, lissage=False):
tracer_evolution(df_clean_TS,["LT_IR_Lithuania"], xlabel ="Période",ylabel = "Taux d'intérêt- Long terme")

In [ ]:
#Pour le moment la série ne semble avoir ni tendance ni saisonalité, essayons d'appliquer un test ADF
result = adfuller(df_clean_TS["LT_IR_Lithuania"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

#La série n'est pas stationnaire
#essayons de faire une différenciation


In [ ]:
LT_IR_Lithuania_diff=df_clean_TS["LT_IR_Lithuania"].diff()
#LT_IR_Latvia_diff.plot(title="Taux d'intérêt Lettonie différencié", figsize=(12, 4))

plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR_Lithuania")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR_Lithuania")
plt.show()

In [ ]:
#Résult
result = adfuller(LT_IR_Lithuania_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
#On rejette à 5% l'hypothèse de non stationarité


In [ ]:
LT_IR_Lithuania_models=evaluate_all_models(LT_IR_Lithuania_diff.dropna(), 4, 4)


In [ ]:
compute_aic_bic(LT_IR_Lithuania_diff,LT_IR_Lithuania_models,0)


In [ ]:
# LT_IR_Lithuania_pred = backcast_arima(df_clean_TS, "LT_IR_Lithuania", (0,1,4), trend='t')
LT_IR_Lithuania_pred = backcast_arima(df_clean_TS, "LT_IR_Lithuania", (0,1,4), trend='t')
LT_IR_Lithuania_pred_final = LT_IR_Lithuania_pred.combine_first(df_clean_TS["LT_IR_Lithuania"])

# Si LT_IR_Lithuania_pred_final est un pd.Series indexé en datetime :
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Lithuania_pred_final.index, 
         LT_IR_Lithuania_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Lithuania — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()


#### LT_IR_Poland<a class="anchor" id="partie213"></a>

In [ ]:
# Le code qui suit sert uniquement à visualiser les ST
# syntaxe: tracer_evolution(df, columns=None, xlabel= None, ylabel=None, start_date=None, end_date=None, log=False, base=None, lissage=False):
tracer_evolution(df_clean_TS, ["LT_IR_Poland"], xlabel="Période", ylabel="Taux d'intérêt- court terme")

In [ ]:
#La série semble avoir une tendance décroissante, essayons d'appliquer un test ADF
result = adfuller(df_clean_TS["LT_IR_Poland"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

# La série n'est pas stationnaire
# Essayons de faire une différenciation

In [ ]:
LT_IR_Poland_diff = df_clean_TS["LT_IR_Poland"].diff()
LT_IR_Poland_diff.plot(title="Taux d'intérêt Pologne différencié", figsize=(12, 4))
# La série semble centrée en 0, mais avec une légère tendance à la hausse

In [ ]:
# Résultat du test ADF sur la série différenciée
result = adfuller(LT_IR_Poland_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
# On rejette à 5% l'hypothèse de non stationnarité

In [ ]:

plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Poland_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR Poland")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Poland_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR Poland")
plt.show()

In [ ]:
LT_IR_Poland_models = evaluate_all_models(LT_IR_Poland_diff.dropna(), 1, 2)
#Les ordres conservés: (1,2)

In [ ]:
compute_aic_bic(LT_IR_Poland_diff, LT_IR_Poland_models, 0)
#Modèle retenu selon le criètere BIC


In [ ]:
# On essaie d'appliquer la fonction backcast
LT_IR_Poland_pred = backcast_arima(df_clean_TS, "LT_IR_Poland", (1,1,0), trend='t')
LT_IR_Poland_pred_final = LT_IR_Poland_pred.combine_first(df_clean_TS["LT_IR_Poland"])
LT_IR_Poland_pred_final

In [ ]:
# Si LT_IR_Poland_pred_final est un pd.Series indexé en datetime :
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Poland_pred_final.index, 
         LT_IR_Poland_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Poland — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### LT_IR_Slovakia<a class="anchor" id="partie214"></a>

In [ ]:
# Le code qui suit sert uniquement à visualiser les ST
# syntaxe: tracer_evolution(df, columns=None, xlabel= None, ylabel=None, start_date=None, end_date=None, log=False, base=None, lissage=False):
tracer_evolution(df_clean_TS, ["LT_IR_Slovakia"], xlabel="Période", ylabel="Taux d'intérêt- court terme")

In [ ]:
# Pour le moment la série semble avoir une tendane décroissante. Appliquons un ADF
result = adfuller(df_clean_TS["LT_IR_Slovakia"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
# La série n'est pas stationnaire
# Essayons de faire une différenciation

In [ ]:
LT_IR_Slovakia_diff = df_clean_TS["LT_IR_Slovakia"].diff()
LT_IR_Slovakia_diff.plot(title="Taux d'intérêt Slovaquie différencié", figsize=(12, 4))


In [ ]:
# Résultat du test ADF sur la série différenciée
result = adfuller(LT_IR_Slovakia_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
# On rejette à 5% l'hypothèse de non stationnarité

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Slovakia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR_Slovakia")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Slovakia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR_Slovakia")
plt.show()

In [ ]:
#modèles retenus: (1,1)
LT_IR_Slovakia_models = evaluate_all_models(LT_IR_Slovakia_diff.dropna(), 1, 1)


In [ ]:
compute_aic_bic(LT_IR_Slovakia_diff, LT_IR_Slovakia_models, 0)

In [ ]:
# On essaie d'appliquer la fonction précédente
# LT_IR_Slovakia_pred = backcast_arima(df_clean_TS, "LT_IR_Slovakia", (0,1,1))
LT_IR_Slovakia_pred = backcast_arima(df_clean_TS, "LT_IR_Slovakia", (0,1,1), trend='t')
LT_IR_Slovakia_pred_final = LT_IR_Slovakia_pred.combine_first(df_clean_TS["LT_IR_Slovakia"])
LT_IR_Slovakia_pred_final

In [ ]:
# Si LT_IR_Slovakia_pred_final est un pd.Series indexé en datetime :
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Slovakia_pred_final.index, 
         LT_IR_Slovakia_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Slovakia — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### LT_IR_Czechia<a class="anchor" id="partie215"></a>
Attention: aucun modèle ne passe tous les tests malheuresement, on a gardé celui obtenu visuellement à partir de ACF et PACF

In [ ]:
tracer_evolution(df_clean_TS, ["LT_IR_Czechia"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["LT_IR_Czechia"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
LT_IR_Czechia_diff = df_clean_TS["LT_IR_Czechia"].diff()
LT_IR_Czechia_diff.plot(title="Taux d'intérêt Chéquie différencié", figsize=(12, 4))

result = adfuller(LT_IR_Czechia_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")


In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Czechia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR_Czechia")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Czechia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR_Czechia")
plt.show()

In [ ]:
LT_IR_Czechia_models = evaluate_all_models(LT_IR_Czechia_diff.dropna(), 5, 0)
# Problème: aucun modèle n'est retenu à la fin, on va tout de même garder le modèle (5,1,0)

In [ ]:

LT_IR_Czechia_pred = backcast_arima(df_clean_TS, "LT_IR_Czechia", (5,1,0), trend='t')
LT_IR_Czechia_pred_final = LT_IR_Czechia_pred.combine_first(df_clean_TS["LT_IR_Czechia"])
LT_IR_Czechia_pred_final

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Czechia_pred_final.index, 
         LT_IR_Czechia_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Czechia — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### LT_IR_Hungary<a class="anchor" id="partie216"></a>


In [ ]:
tracer_evolution(df_clean_TS, ["LT_IR_Hungary"], xlabel="Période", ylabel="Taux d'intérêt- court terme")

result = adfuller(df_clean_TS["LT_IR_Hungary"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
LT_IR_Hungary_diff = df_clean_TS["LT_IR_Hungary"].diff()
LT_IR_Hungary_diff.plot(title="Taux d'intérêt Hongrie différencié", figsize=(12, 4))


result = adfuller(LT_IR_Hungary_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Hungary_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR_Hungary")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Hungary_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR_Hungary")
plt.show()

In [ ]:
LT_IR_Hungary_models = evaluate_all_models(LT_IR_Hungary_diff.dropna(), 3, 1)


In [ ]:
compute_aic_bic(LT_IR_Hungary_diff, LT_IR_Hungary_models, 0)


In [ ]:
LT_IR_Hungary_pred = backcast_arima(df_clean_TS, "LT_IR_Hungary", (1,1,0), trend='t')
LT_IR_Hungary_pred_final = LT_IR_Hungary_pred.combine_first(df_clean_TS["LT_IR_Hungary"])
LT_IR_Hungary_pred_final

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Hungary_pred_final.index, 
         LT_IR_Hungary_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Hungary — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### LT_IR_Iceland<a class="anchor" id="partie217"></a>
Ceci est à compléter.... il faut modifier la fonction du backcast, mais cette fois-ci pour faire du forecast

In [ ]:
tracer_evolution(df_clean_TS, ["LT_IR_Iceland"], xlabel="Période", ylabel="Taux d'intérêt- court terme")


#### LT_IR_Greece<a class="anchor" id="partie218"></a>
rq: de loin la meilleure prédiction

In [ ]:
tracer_evolution(df_clean_TS, ["LT_IR_Greece"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["LT_IR_Greece"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
LT_IR_Greece_diff = df_clean_TS["LT_IR_Greece"].diff()
result = adfuller(LT_IR_Greece_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")


In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(LT_IR_Greece_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du LT_IR_Greece")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(LT_IR_Greece_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du LT_IR_Greece")
plt.show()

In [ ]:
LT_IR_Greece_models = evaluate_all_models(LT_IR_Greece_diff.dropna(), 1, 9)


In [ ]:
compute_aic_bic(LT_IR_Greece_diff, LT_IR_Greece_models, 0)


In [ ]:
LT_IR_Greece_pred = backcast_arima(df_clean_TS, "LT_IR_Greece", (2,1,4), trend='t')
LT_IR_Greece_pred_final = LT_IR_Greece_pred.combine_first(df_clean_TS["LT_IR_Greece"])
LT_IR_Greece_pred_final

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(LT_IR_Greece_pred_final.index, 
         LT_IR_Greece_pred_final.values, 
         marker='o', linestyle='-')
plt.title("LT_IR_Greece — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

### WH <a class="anchor" id="partie22"></a>

Il y a une composante saisonnière dans chacune des séries. Il faudra modifier le programme de base

#### WH_Bulgaria <a class="anchor" id="partie221"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["WH_Bulgaria"], xlabel="Période", ylabel="Taux d'intérêt- court terme")

result = adfuller(df_clean_TS["WH_Bulgaria"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
WH_Bulgaria_diff = df_clean_TS["WH_Bulgaria"].diff()
WH_Bulgaria_diff.plot(title="Taux d'intérêt Bulgarie différencié", figsize=(12, 4)) 

result = adfuller(WH_Bulgaria_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(WH_Bulgaria_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du WH_Bulgaria")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(WH_Bulgaria_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du WH_Bulgaria")
plt.show()

In [ ]:
WH_Bulgaria_models = evaluate_all_models(WH_Bulgaria_diff.dropna(), 3, 8)

In [ ]:
compute_aic_bic(WH_Bulgaria_diff, WH_Bulgaria_models, 0)


In [ ]:
WH_Bulgaria_pred = backcast_arima(df_clean_TS, "WH_Bulgaria", (3,1,0), trend='t')
WH_Bulgaria_pred_final = WH_Bulgaria_pred.combine_first(df_clean_TS["WH_Bulgaria"])

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(WH_Bulgaria_pred_final.index, WH_Bulgaria_pred_final.values, marker='o', linestyle='-')
plt.title("WH_Bulgaria — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### WH_Lithuania <a class="anchor" id="partie222"></a>
ATTENTION, la série différenciée ne rejette pas le test ADF
Cette série a une composante saisonnière: on laisse tomber pour le moment

In [ ]:
tracer_evolution(df_clean_TS, ["WH_Lithuania"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["WH_Lithuania"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
WH_Lithuania_diff = df_clean_TS["WH_Lithuania"].shift(4)  # saisonnalité
WH_Lithuania_diff = df_clean_TS["WH_Lithuania"] - WH_Lithuania_diff
WH_Lithuania_diff = WH_Lithuania_diff - WH_Lithuania_diff.shift(1)  # tendance


WH_Lithuania_diff.plot(title="Taux d'intérêt Lituanie différencié", figsize=(12, 4)) 

result = adfuller(WH_Lithuania_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(WH_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du WH_Lithuania")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(WH_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du WH_Lithuania")
plt.show()

In [ ]:
WH_Lithuania_models = evaluate_all_models(WH_Lithuania_diff.dropna(), 13, 16)


#### WH_Latvia <a class="anchor" id="partie223"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["WH_Latvia"], xlabel="Période", ylabel="Taux d'intérêt- court terme")

result = adfuller(df_clean_TS["WH_Latvia"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
WH_Latvia_diff = df_clean_TS["WH_Latvia"].diff()
WH_Latvia_diff.plot(title="Taux d'intérêt Lettonie différencié", figsize=(12, 4)) 

result = adfuller(WH_Latvia_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(WH_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du WH_Latvia")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(WH_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du WH_Latvia")
plt.show()

#### WH_Slovakia <a class="anchor" id="partie224"></a>

#### WH_Poland <a class="anchor" id="partie225"></a>

#### WH_Czechia <a class="anchor" id="partie226"></a>

#### WH_Romania <a class="anchor" id="partie227"></a>

### ST_IR <a class="anchor" id="partie23"></a>

#### ST_IR_Switzerland <a class="anchor" id="partie231"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["ST_IR_Switzerland"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["ST_IR_Switzerland"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
ST_IR_Switzerland_diff = df_clean_TS["ST_IR_Switzerland"].diff()
ST_IR_Switzerland_diff.plot(title="Taux d'intérêt Suisse différencié", figsize=(12, 4))
result = adfuller(ST_IR_Switzerland_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(ST_IR_Switzerland_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du ST_IR_Switzerland")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(ST_IR_Switzerland_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du ST_IR_Switzerland")
plt.show()

In [ ]:
ST_IR_Switzerland_models = evaluate_all_models(ST_IR_Switzerland_diff.dropna(), 1, 2)


In [ ]:
compute_aic_bic(ST_IR_Switzerland_diff, ST_IR_Switzerland_models, 0)


In [ ]:
ST_IR_Switzerland_pred = backcast_arima(df_clean_TS, "ST_IR_Switzerland", (1,1,0), trend='t')
ST_IR_Switzerland_pred_final = ST_IR_Switzerland_pred.combine_first(df_clean_TS["ST_IR_Switzerland"])

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(ST_IR_Switzerland_pred_final.index, ST_IR_Switzerland_pred_final.values, marker='o', linestyle='-')
plt.title("ST_IR_Switzerland — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### ST_IR_Lithuania <a class="anchor" id="partie232"></a>

In [ ]:


tracer_evolution(df_clean_TS, ["ST_IR_Lithuania"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["ST_IR_Lithuania"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
ST_IR_Lithuania_diff = df_clean_TS["ST_IR_Lithuania"].diff()
ST_IR_Lithuania_diff.plot(title="Taux d'intérêt Lituanie différencié", figsize=(12, 4))

result = adfuller(ST_IR_Lithuania_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(ST_IR_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du ST_IR_Lithuania")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(ST_IR_Lithuania_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du ST_IR_Lithuania")
plt.show()


In [ ]:
ST_IR_Lithuania_models = evaluate_all_models(ST_IR_Lithuania_diff.dropna(), 2, 1)


In [ ]:
compute_aic_bic(ST_IR_Lithuania_diff, ST_IR_Lithuania_models, 0)


In [ ]:
ST_IR_Lithuania_pred = backcast_arima(df_clean_TS, "ST_IR_Lithuania", (0,1,1), trend='t')
ST_IR_Lithuania_pred_final = ST_IR_Lithuania_pred.combine_first(df_clean_TS["ST_IR_Lithuania"])


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(ST_IR_Lithuania_pred_final.index, ST_IR_Lithuania_pred_final.values, marker='o', linestyle='-')
plt.title("ST_IR_Lithuania — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### ST_IR_Luxembourg <a class="anchor" id="partie233"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["ST_IR_Luxembourg"], xlabel="Période", ylabel="Taux d'intérêt- court terme")
result = adfuller(df_clean_TS["ST_IR_Luxembourg"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
ST_IR_Luxembourg_diff = df_clean_TS["ST_IR_Luxembourg"].diff()
ST_IR_Luxembourg_diff.plot(title="Taux d'intérêt Luxembourg différencié", figsize=(12, 4))

result = adfuller(ST_IR_Luxembourg_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(ST_IR_Luxembourg_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du ST_IR_Luxembourg")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(ST_IR_Luxembourg_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du ST_IR_Luxembourg")
plt.show()

In [ ]:
ST_IR_Luxembourg_models = evaluate_all_models(ST_IR_Luxembourg_diff.dropna(), 5, 2)


In [ ]:
compute_aic_bic(ST_IR_Luxembourg_diff, ST_IR_Luxembourg_models, 0)


In [ ]:
ST_IR_Luxembourg_pred = backcast_arima(df_clean_TS, "ST_IR_Luxembourg", (1,1,0), trend='t')
ST_IR_Luxembourg_pred_final = ST_IR_Luxembourg_pred.combine_first(df_clean_TS["ST_IR_Luxembourg"])


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(ST_IR_Luxembourg_pred_final.index, ST_IR_Luxembourg_pred_final.values, marker='o', linestyle='-')
plt.title("ST_IR_Luxembourg — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

#### ST_IR_Latvia <a class="anchor" id="partie233"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["ST_IR_Latvia"], xlabel="Période", ylabel="Taux d'intérêt- court terme")


result = adfuller(df_clean_TS["ST_IR_Latvia"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
ST_IR_Latvia_diff = df_clean_TS["ST_IR_Latvia"].diff()
ST_IR_Latvia_diff.plot(title="Taux d'intérêt Lettonie différencié", figsize=(12, 4))

result = adfuller(ST_IR_Latvia_diff.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
plt.figure(figsize=(14,6))
tsaplots.plot_acf(ST_IR_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation du ST_IR_Latvia")
plt.show()

plt.figure(figsize=(14,6))
tsaplots.plot_pacf(ST_IR_Latvia_diff.dropna(), lags=30)
plt.title("Fonction d'autocorrélation partielle du ST_IR_Latvia")
plt.show()

In [ ]:
ST_IR_Latvia_models = evaluate_all_models(ST_IR_Latvia_diff.dropna(), 5, 5)


In [ ]:
compute_aic_bic(ST_IR_Latvia_diff, ST_IR_Latvia_models, 0)


In [ ]:
ST_IR_Latvia_pred = backcast_arima(df_clean_TS, "ST_IR_Latvia", (1,1,2), trend='t')
ST_IR_Latvia_pred_final = ST_IR_Latvia_pred.combine_first(df_clean_TS["ST_IR_Latvia"])


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(ST_IR_Latvia_pred_final.index, ST_IR_Latvia_pred_final.values, marker='o', linestyle='-')
plt.title("ST_IR_Latvia — série complète backcastée + observée")
plt.xlabel("Date")
plt.ylabel("Taux (%)")
plt.grid(True)
plt.tight_layout()
plt.show()

### P_Growth <a class="anchor" id="partie24"></a>

#### P_Growth_Italy <a class="anchor" id="partie241"></a>

In [ ]:
tracer_evolution(df_clean_TS, ["P_Growth_France"], xlabel="Période", ylabel="Croissance potentielle")

result = adfuller(df_clean_TS["P_Growth_Italy"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [74]:
from numpy.polynomial import Polynomial

def forecast_polynomial(df, col, degree=3):
    """
    Remplit les NaN à la fin d'une série par une régression polynomiale.

    Paramètres :
    - df : DataFrame contenant la série
    - col : nom de la colonne cible
    - degree : degré du polynôme à ajuster

    Retour :
    - pd.Series complète avec prédiction sur les NaN de fin
    """
    y = df[col]
    valid_y = y.dropna()
    
    if len(valid_y) < degree + 1:
        raise ValueError("Pas assez de points pour ajuster un polynôme.")

    # Indices des points observés
    x_obs = np.arange(len(valid_y))
    y_obs = valid_y.values

    # Ajustement polynômial
    poly = Polynomial.fit(x_obs, y_obs, deg=degree)

    # Indices de toute la série
    x_full = np.arange(len(y))
    y_pred_full = poly(x_full)

    # Création de la série prédite
    y_pred_series = pd.Series(y_pred_full, index=y.index, name=col)

    # Remplissage uniquement des NaN à la fin
    return y.combine_first(y_pred_series)


In [ ]:
P_Growth_France_forecast= forecast_polynomial(df_clean_TS, "P_Growth_France")


plt.figure(figsize=(10, 4))
plt.plot(P_Growth_France_forecast.index, 
         P_Growth_France_forecast.values, 
         marker='o', linestyle='-')
plt.title("P_Growth_France — série complétée par régression polynomiale")
plt.xlabel("Date")
plt.ylabel("Croissance potentielle")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
P_Growth_Italy_diff = df_clean_TS["P_Growth_Italy"].diff()
P_Growth_Italy_diff = P_Growth_Italy_diff.diff()
P_Growth_Italy_diff=P_Growth_Italy_diff.diff()

P_Growth_Italy_diff.plot(title="Croissance potentielle Italie différenciée", figsize=(12, 4))

### Visualisation progressive du nettoyage<a class="anchor" id="partie3"></a>

Pour visualier le nettoyage progressif des fonctions

In [ ]:
#Pour visualiser le nettoyage progressif des données
df_clean_missing_binary = df_clean.isnull().astype(int)

# Définir la figure avec un affichage interactif
fig, ax = plt.subplots(figsize=(50, 80))

# Utiliser seaborn pour une meilleure visualisation sans grille
sns.heatmap(df_clean_missing_binary, cmap="OrRd", cbar=False, linewidths=0, ax=ax, square=False)

# Ajouter un titre
ax.set_title("Matrice des valeurs manquantes (Rouge = Manquant)")

# Rotation des labels pour une meilleure lisibilité
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Afficher la figure
plt.show()


In [ ]:
#Pourcentage de données manquantes par variable
percentage_missing_clean=((df_clean_missing_binary.sum())/df_clean_missing_binary.shape[0])*100

#On ne garde que les colonnes ayant des missing values
percentage_missing_clean=percentage_missing_clean[percentage_missing_clean>0]
percentage_missing_clean = percentage_missing_clean.sort_values(ascending=False)
percentage_missing_clean